# Robustness and Plasticity in Regulatory Networks

* [Introduction](./RPRN-Introduction.ipynb)
* [BoolNet](./RPRN-BoolNet.ipynb)
* [Functions](#Functions)
    * [Overexpression and KnockOuts](#Overexpression-and-KnockOuts)
    * [Fixed environments](#Fixed-environments)
    * [Truth tables](#Truth-tables)
* [Updating](./RPRN-Updating.ipynb)
* [States](./RPRN-States.ipynb)
* [Appendix](./RPRN-Appendix.ipynb)

# Functions

Functions recapitulate the regulatory interactions and determine the dynamic of the Boolean regulatory network. They are limited by the topology, as nodes can only be directly influeced by their regulators. Changing the functions of a network can create a whole new network. This new network may share some attractors -or cell types- with the original network, but it may also lose or gain other attractors.

The changes in the functions of a network can be associated with multiple biological phenomena. Experiments, environmental factors, evolution, epigenetics and the intrinsic flexibility of the regulatory mechanisms all alter the iteractions of the regulatory network of a biological system.

In this tutorial we will use some of the existing BoolNet functions to study the effect of perturbing the network functions. We will also use some of the labeling functions we discussed in the [BoolNet]((./RPRN-BoolNet.ipynb)) introduction.

In [1]:
#Uncomment next line if you haven't installed BoolNet
#install.packages("BoolNet", repos='http://cran.us.r-project.org')
library(BoolNet)
source("./BoolNet-extensions.R")

## Overexpression and KnockOuts

A lot of experiments  inhibit or overexpress genes, with is equivalent to fixing the functions to 0 or 1. Studing this perturbations is useful for validating the network with known gain and loss of function experiments. It can also be used to predict the effect of difficult experiments like letal or conditional mutants.

First, we need to know how our wild type network will behave. We will use the Th17/iTreg network.

In [2]:
net <- loadNetwork("minTh17iTreg.txt")
WT.attr <- getAttractors(net)

However, analysing this structure can be complicated, we will label them and convert them to a data frame. Here we will use the __attractor2dataframe()__ and __labelAttractors()__ functions.

First we will declare our labelling rules.

In [3]:
labels.rules <- data.frame(
    labels = c('Th0', 'Th17', 'Treg', 'IL10+', 'TGFB+', 'RORGT+'),
    rules  = c('!(RORGT | FOXP3 | TGFB | IL10)', 
            'RORGT & STAT3', 
            'FOXP3 & TGFB', 
            'IL10', 
            'TGFB & ! (RORGT | FOXP3)', 
            'RORGT & ! STAT3' ),
    stringsAsFactors = FALSE
)
labels.rules

,labels,rules
1,Th0,!(RORGT | FOXP3 | TGFB | IL10)
2,Th17,RORGT & STAT3
3,Treg,FOXP3 & TGFB
4,IL10+,IL10
5,TGFB+,TGFB & ! (RORGT | FOXP3)
6,RORGT+,RORGT & ! STAT3


Next we will create a data frame with all the information.

In [4]:
WT.attr.df <- attractor2dataframe(WT.attr) 
WT.labels <- labelAttractors(WT.attr, net$genes, labels.rules$labels, labels.rules$rules)
WT.attr.df$label <- sapply(WT.labels, function(label) {
    paste(as.character(label), collapse='/')
})
WT.attr.df

,involvedStates,basinSize,label
1,0,27,Th0
2,1,2,Th0
3,4,14,Th0
4,16,13,TGFB+
5,48,8,IL10+TGFB+
6,65,35,Th0
7,68,8,Th0
8,89,13,Treg
9,112,3,IL10+TGFB+
10,121,1,TregIL10+


Now that we know the behaivor of the WT network we can begin perturbing it. 

We will do a knock-out experiment, where the value of STAT3 will always be zero. This wpuld be equivalent to deleting the STAT3 gene or inhibiting all the signaling pathways that end in STAT3 activation.

We can fix the value of the node in the original network or we can create a second mutant network. 

In [6]:
KO.net <- fixGenes(net, "STAT3", 0)
KO.net

Boolean network with 10 genes

Involved genes:
IL2 RORGT STAT3 FOXP3 TGFB IL10 IL2e IL21e TGFBe IL10e

Transition functions:
IL2 = (IL2e | (IL2 &  ! FOXP3)) &  ! (STAT3 | (IL10 & ! FOXP3))
RORGT = (STAT3 & TGFB) &  ! FOXP3
STAT3 = (IL21e | STAT3 | RORGT) &  ! (IL10 | IL2)
FOXP3 = (IL2 & (TGFB | FOXP3)) &  ! (STAT3 | RORGT)
TGFB = TGFBe | ((TGFB | FOXP3) &  ! STAT3 )
IL10 = IL10e | (IL10 & (STAT3 | TGFB))
IL2e = IL2e
IL21e = IL21e
TGFBe = TGFBe
IL10e = IL10e

Knocked-out and over-expressed genes:
STAT3 = 0

When doing this be careful of checking the value of the fixed genes to avoid mistakes. You can see this in network$fixed, if the value is -1 BoolNEt will use the original function, if the value is 0 the node value will always be 0, and if the value is 1 the node value will always be 1.

In [8]:
KO.net$fixed

IL2 RORGT STAT3 FOXP3  TGFB  IL10  IL2e IL21e TGFBe IL10e 
   -1    -1     0    -1    -1    -1    -1    -1    -1    -1

Now we will determine the attractors, label them, and save them to a dataframe.

In [9]:
KO.attr <- getAttractors(KO.net)
KO.attr.df <- attractor2dataframe(KO.attr) 
KO.labels <- labelAttractors(KO.attr, net$genes, labels.rules$labels, labels.rules$rules)
KO.attr.df$label <- sapply(KO.labels, function(label) {
    paste(as.character(label), collapse='/')
})
KO.attr.df

,involvedStates,basinSize,label
1,0,6,Th0
2,1,2,Th0
3,16,16,TGFB+
4,48,8,IL10+TGFB+
5,65,8,Th0
6,89,16,Treg
7,112,3,IL10+TGFB+
8,121,1,TregIL10+
9,128,6,Th0
10,129,2,Th0


We  can also simulate overexpressions, where the value of STAT3 will always be 1. This would be equivalent to having the STAT3 pathway constitutionally active.

We will create the network, determine the attractors, label them and present them as a dataframe.

In [11]:
Over.net <- fixGenes(net, "STAT3", 1)
Over.attr <- getAttractors(Over.net)
Over.attr.df <- attractor2dataframe(Over.attr) 
Over.labels <- labelAttractors(Over.attr, net$genes, labels.rules$labels, labels.rules$rules)
Over.attr.df$label <- sapply(Over.labels, function(label) {
    paste(as.character(label), collapse='/')
})
Over.attr.df

,involvedStates,basinSize,label
1,4,16,Th0
2,36,16,IL10+
3,68,16,Th0
4,100,16,IL10+
5,132,16,Th0
6,164,16,IL10+
7,196,16,Th0
8,228,16,IL10+
9,278,16,Th17
10,310,16,Th17IL10+


Now we can compare the three networks. Lets begin by seeing the cell types that where recovered in each network.

In [12]:
unique(WT.attr.df$label)
unique(KO.attr.df$label)
unique(Over.attr.df$label)

[1] "Th0"                  "TGFB+"                "IL10+TGFB+"          
 [4] "Treg"                 "TregIL10+"            "Th17"                
 [7] "IL10+"                "IL10+TGFB+/TregIL10+" "Th0/Th0"             
[10] "RORGT+/TGFB+"

[1] "Th0"                  "TGFB+"                "IL10+TGFB+"          
[4] "Treg"                 "TregIL10+"            "IL10+"               
[7] "IL10+TGFB+/TregIL10+"

[1] "Th0"       "IL10+"     "Th17"      "Th17IL10+"

We can also see which attractors where won or lost in our mutants. 
In the case of the STAT3$^{KO}$ we lost the Th17 attractor and some cycles.

In [14]:
setdiff(WT.attr.df$label,KO.attr.df$label)
setdiff(KO.attr.df$label,WT.attr.df$label)

[1] "Th17"         "Th0/Th0"      "RORGT+/TGFB+"

character(0)

In the case of the STAT3$^{OverExpression}$ we lost most of the regulatory attractors and some cycles. We also won a Th17IL1

In [15]:
setdiff(WT.attr.df$label,Over.attr.df$label)
setdiff(Over.attr.df$label,WT.attr.df$label)

[1] "TGFB+"                "IL10+TGFB+"           "Treg"                
[4] "TregIL10+"            "IL10+TGFB+/TregIL10+" "Th0/Th0"             
[7] "RORGT+/TGFB+"

[1] "Th17IL10+"

## Fixed environments

The state of a network usually depends in external factors, that can be modeles as inputs. It is possible to simulate the change from one environment to an other by fixing the input nodes to simulate the different environments\cite{Thieffry}. In this section we will study the effect of permanetly changing the environment of a cell type, we will discuss the effect of [transient perturbations later](./RPRN-States.ipynb).

## Truth tables

Trough evolution the regulatory interactions of an organism can change. For example, the regulatory sequence of a gene can be altered or a new element can be added (or lost)\cite{Carlos???}. However, this changes can also occur during the life of an organism: changes in the epigenetic marks, disordered domain proteins, and alternate spliccing can alter the regulatory functions\cite{Newman2014}. Finally, it is possible that there are mistakes in the construction of the network.

This changes usually affect only part of the regulatory function. Verifing the robustness of the attractors in response to small changes in the functions is fundamental for validating the network and for understanding the evolvability of biological systems.

# Next

* [Introduction](./RPRN-Introduction.ipynb)
* [BoolNet](./RPRN-BoolNet.ipynb)
* Functions
* [Updating](./RPRN-Updating.ipynb)
* [States](./RPRN-States.ipynb)
* [Appendix](./RPRN-Appendix.ipynb)